In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('./отзывы за лето.xls')
df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [3]:
max_words = 1000
max_len = 100
num_classes = 6

epochs = 20
batch_size = 512
print_batch_n = 100

In [4]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split

In [5]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)

In [6]:
train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=17)
test_df, val_df = train_test_split(test_val_df, test_size=0.5, random_state=17)

In [7]:
train_corpus = " ".join(train_df['Content'])
train_corpus = train_corpus.lower()

In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Никита\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
tokens_filtered = [word for word in tokens if word.isalnum()]

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [10]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [11]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [12]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in train_df['Content']], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test_df['Content']], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in val_df['Content']], dtype=np.int32)

# Keras model

In [13]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  
from keras.utils import np_utils

In [14]:
y_train = keras.utils.np_utils.to_categorical(train_df['Rating'], num_classes)
y_test = keras.utils.np_utils.to_categorical(test_df['Rating'], num_classes)
y_val = keras.utils.np_utils.to_categorical(val_df['Rating'], num_classes)

In [15]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 3s 104ms/step - loss: 1.4142 - accuracy: 0.6759 - val_loss: 1.1631 - val_accuracy: 0.6814
Epoch 2/20
26/26 [==============================] - 3s 99ms/step - loss: 0.9536 - accuracy: 0.7078 - val_loss: 0.8688 - val_accuracy: 0.6814


In [18]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 0s 26ms/step - loss: 0.8022 - accuracy: 0.7102


Test score: 0.8022122383117676
Test accuracy: 0.7102290987968445


In [19]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 0s 25ms/step


### Embedding слой 

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['Content'])
vocab_size = len(tokenizer.word_index) + 1

In [22]:
embeddings_index = {}
with open('tayga_1_2.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


In [23]:
# Матрица эмбеддингов

embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 270 words (9609 misses)


In [24]:
# Сеть

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[embedding_matrix],
    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [25]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [26]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 4s 153ms/step - loss: 1.7421 - accuracy: 0.6805 - val_loss: 1.6694 - val_accuracy: 0.6849
Epoch 2/20
26/26 [==============================] - 4s 143ms/step - loss: 1.5680 - accuracy: 0.7135 - val_loss: 1.4421 - val_accuracy: 0.6856


In [27]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 0s 45ms/step - loss: 1.4222 - accuracy: 0.7193


Test score: 1.4221816062927246
Test accuracy: 0.7192642688751221


In [28]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 0s 43ms/step
